## Punto 12
En el siguiente experimento compararemos la eficacia de los siguientes métodos de resolución de sistemas de ecuaciones lineales: eliminación gaussiana con y sin pivote, eliminación de Gauss-Jordan con y sin pivote, descomposición de Doolittle con y sin pivote, y descomposición de Choleski. Para la comparación usaremos tres sistemas de ecuaciones a partir de tres matrices con diferentes características que nos ayudarán a realizar un mejor anális de la eficacia de los métodos anteriores. Empecemos primero por importar las librerías y declarar las funciones a usar:

In [1]:
import numpy as np
import math 
import time

In [2]:
def gaussElimin(A,B):
    "Resuelve Ax=B por eliminación gaussiana."
    a=A.copy()
    b=B.copy()
    n = len(a[1])
    it=0
    #Fase de eliminación
    for k in range(0,n-1):
        for i in range(k+1,n):
            if a[k,k]==0:
                return "Un elemento diagonal es cero, no se puede realizar eliminación gaussiana."
            lam=a[i,k]/a[k,k]
            a[i]=a[i] - lam*a[k]
            b[i]=b[i] - lam*b[k]
            it+=1
    #Fase de sustitución
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(a[k,k+1:n],b[k+1:n]))/a[k,k]
        it+=1
    return b,it

def gaussJordan(A,B):
    "Resuelve Ax=B por Gauss-Jordan."
    a=A.copy()
    b=B.copy()
    n = len(a[1])
    it=0
    #Fase de eliminación
    for k in range(n-1):
        for i in range(k+1,n):
            if a[k,k]==0:
                return "Un elemento diagonal es cero, no se puede realizar Gauss-Jordan."
            lam=a[i,k]/a[k,k]
            a[i]=a[i]-lam*a[k]
            b[i]=b[i]-lam*b[k]
            it+=1
    for k in range(1,n):
        for i in range(n):
            if a[i,k]!=a[k,k]:
                lam=a[i,k]/a[k,k]
                a[i]=a[i]-lam*a[k]
                b[i]=b[i]-lam*b[k]
                it+=1
    #Fase de sustitución
    for i in range(n):
        b[i]=b[i]/a[i,i]
        it+=1
    return b,it

def LU(A,B):
    "Resuelve un sistema de ecuaciones por descomposición LU usando el método de Doolittle para una matriz a."
    a=A.copy()
    b=B.copy()
    n = len(a)
    it=0
    for k in range(0,n-1):
        for i in range(k+1,n):
            if a[k,k]==0:
                return "Un elemento diagonal es cero, no se puede realizar la descomposición LU de Doolittle."
            lam=a[i,k]/a[k,k]
            a[i,k+1:n]=a[i,k+1:n] - lam*a[k,k+1:n]
            a[i,k] = lam
            it+=1
    for k in range(1,n):
        b[k] = b[k] - np.dot(a[k,0:k],b[0:k])
        it+=1
    b[n-1] = b[n-1]/a[n-1,n-1]
    for k in range(n-2,-1,-1):
        b[k] = (b[k] - np.dot(a[k,k+1:n],b[k+1:n]))/a[k,k]
        it+=1
    return b,it

def choleski(A,B):
    "Soluciona un sistema de ecuaciones por medio de la descomposición de Choleski."
    a=A.copy()
    b=B.copy()
    n = len(a)
    it=0
    for i in range(n):
        for j in range(n):
            if a[i,j]!=a[j,i]:
                return "La matriz no es simétrica, no se puede realizar la descomposición de Choleski."
        it+=1
    for k in range(n):
        try:
            a[k,k] = math.sqrt(a[k,k]- np.dot(a[k,0:k],a[k,0:k]))
        except ValueError:
            return "La matriz no es definida positiva, no se puede realizar la descomposición de Choleski."
        for i in range(k+1,n):
            a[i,k] = (a[i,k] - np.dot(a[i,0:k],a[k,0:k]))/a[k,k]
            it+=1
    for k in range(1,n): 
        a[0:k,k] = 0.0
        it+=1
    
    n = len(b)
    for k in range(n):
        b[k] = (b[k] - np.dot(a[k,0:k],b[0:k]))/a[k,k]
        it+=1
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(a[k+1:n,k],b[k+1:n]))/a[k,k]
        it+=1
    return b,it

def swapRows(v,i,j):
    "Cambio de filas para pivotear."
    if len(v.shape) == 1:
        v[i],v[j] = v[j],v[i]
    else:
        v[[i,j],:] = v[[j,i],:]

def GaussPivot(A,B,tol=1.0e-12):
    "Resuelve Ax=B por eliminación gaussiana con pivote."
    a=A.copy()
    b=B.copy()
    n = len(b)
    it=0
    #Calculando los factores de escala
    s = np.zeros(n)
    for i in range(n):
        s[i] = max(np.abs(a[i,:]))
        it+=1
        
    for k in range(0,n-1):
        #Intercambio de filas si es necesario
        p = np.argmax(np.abs(a[k:n,k])/s[k:n]) + k
        if abs(a[p,k]) < tol: 
            return "La matriz es singular."
        if p != k:
            swapRows(b,k,p)
            swapRows(s,k,p)
            swapRows(a,k,p)
        it+=1
            
        #Eliminación
        for i in range(k+1,n):
            lam = a[i,k]/a[k,k]
            a[i] = a[i] - lam*a[k]
            b[i] = b[i] - lam*b[k]
            it+=1
            
    if abs(a[n-1,n-1]) < tol: 
        return "La matriz es singular."
        
    #Sustitución
    b[n-1] = b[n-1]/a[n-1,n-1]
    for k in range(n-2,-1,-1):
        b[k] = (b[k] - np.dot(a[k,k+1:n],b[k+1:n]))/a[k,k]
        it+=1
    return b,it

def JordanPivot(A,B,tol=1.0e-12):
    "Resuelve Ax=B por Gauss-Jordan con pivote."
    a=A.copy()
    b=B.copy()
    n = len(b)
    it=0
    #Calculando los factores de escala
    s = np.zeros(n)
    for i in range(n):
        s[i] = max(np.abs(a[i,:]))
        it+=1
        
    for k in range(0,n-1):
        #Intercambio de filas si es necesario
        p = np.argmax(np.abs(a[k:n,k])/s[k:n]) + k
        if abs(a[p,k]) < tol: 
            return "La matriz es singular."
        if p != k:
            swapRows(b,k,p)
            swapRows(s,k,p)
            swapRows(a,k,p)
        it+=1
            
        #Eliminación
        for i in range(k+1,n):
            lam = a[i,k]/a[k,k]
            a[i] = a[i] - lam*a[k]
            b[i] = b[i] - lam*b[k]
            it+=1
    if abs(a[n-1,n-1]) < tol: 
        return "La matriz es singular."
        
    for k in range(1,n):
        for i in range(n):
            if a[i,k]!=a[k,k]:
                lam=a[i,k]/a[k,k]
                a[i]=a[i]-lam*a[k]
                b[i]=b[i]-lam*b[k]
            it+=1
    
    #Fase de sustitución
    for i in range(n):
        b[i]=b[i]/a[i,i]
        it+=1
    return b,it

def LUPivot(A,B,tol=1.0e-9):
    "Resuelve un sistema de ecuaciones por el método de Doolittle con pívote."
    a=A.copy()
    b=B.copy()
    n = len(a)
    it=0
    seq = np.array(range(n))
    
    #Calculando los factores de escala
    s = np.zeros((n))
    for i in range(n):
        s[i] = max(abs(a[i,:]))
        it+=1
        
    for k in range(0,n-1):
        #Intercambio de filas si es necesario
        p = np.argmax(np.abs(a[k:n,k])/s[k:n]) + k
        if abs(a[p,k]) < tol: 
            return "La matriz es singular."
        if p != k:
            swapRows(s,k,p)
            swapRows(a,k,p)
            swapRows(seq,k,p)
        it+=1
        
        #Eliminación
        for i in range(k+1,n):
            if a[i,k] != 0.0:
                lam = a[i,k]/a[k,k]
                a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n]
                a[i,k] = lam
            it+=1
    #Rearreglando el vector b
    x = b.copy()
    for i in range(n):
        x[i] = b[seq[i]]
        it+=1
        
    #Solución
    for k in range(1,n):
        x[k] = x[k] - np.dot(a[k,0:k],x[0:k])
        it+=1
    x[n-1] = x[n-1]/a[n-1,n-1]
    for k in range(n-2,-1,-1):
        x[k] = (x[k] - np.dot(a[k,k+1:n],x[k+1:n]))/a[k,k]
        it+=1
    return x,it

def res(f):
    "Muestra la respuesta de un sistema de ecuaciones, el tiempo tomado y el número de iteraciones para los métodos dados."
    start=time.time()
    x,it=f
    end=time.time()
    print("La solución del sistema es:")
    for i in range(3):
        print("x",i+1,"=",x[i])
    print("Se realizaron",it,"iteraciones en",end-start,"segundos.")

Usaremos las siguientes matrices:
$$
A=\begin{pmatrix}
4 & -2 & 2\\
-2 &  1 & 3\\
2 & -2 & 2
  \end{pmatrix}
\qquad
B=\begin{pmatrix}
9 & -6 & 6\\
-6 &  5 & -1\\
6 & -1 & 15
  \end{pmatrix}
\qquad
C=\begin{pmatrix}
9 & -6 & 6\\
-6 &  5 & -1\\
6 & -1 & 12
  \end{pmatrix}
$$
Los sistemas a analizar serán $Ax=d$, $Bx=d$ y $Cx=d$, donde $x=(1,2,3)^T$. Declaremos entonces las matrices a trabajar: 

In [3]:
A=np.array([[4,-2,2],[-2,1,3],[2,-2,2]],dtype=float)
B=np.array([[9,-6,6],[-6,5,-1],[6,-1,15]],dtype=float)
C=np.array([[9,-6,6],[-6,5,-1],[6,-1,12]],dtype=float)
d=np.array([1,2,3],dtype=float)

Considerando el sistema $Ax=d$ podemos ver que los métodos sin pivote y la descomposición de Choleski fracasan. En el caso de los métodos de eliminación y la descomposición de Doolittle, al realizar la primera eliminación un elemento diagonal se vuelve cero y como no se puede pivotear, el método no puede continuar. No es posible realizar la descomposición de Choleski puesto que la matriz $A$ no es simétrica. Comprobemos lo anterior:

In [4]:
print(gaussElimin(A,d))
print(gaussJordan(A,d))
print(LU(A,d))
print(choleski(A,d))

Un elemento diagonal es cero, no se puede realizar eliminación gaussiana.
Un elemento diagonal es cero, no se puede realizar Gauss-Jordan.
Un elemento diagonal es cero, no se puede realizar la descomposición LU de Doolittle.
La matriz no es simétrica, no se puede realizar la descomposición de Choleski.


Probemos ahora los métodos con pívote:

In [5]:
res(GaussPivot(A,d))

La solución del sistema es:
x 1 = -1.0
x 2 = -1.875
x 3 = 0.625
Se realizaron 10 iteraciones en 1.6689300537109375e-06 segundos.


In [6]:
res(JordanPivot(A,d))

La solución del sistema es:
x 1 = -1.0
x 2 = -1.875
x 3 = 0.625
Se realizaron 17 iteraciones en 1.430511474609375e-06 segundos.


In [7]:
res(LUPivot(A,d))

La solución del sistema es:
x 1 = -1.0
x 2 = -1.875
x 3 = 0.625
Se realizaron 15 iteraciones en 9.5367431640625e-07 segundos.


Podemos ver los tres métodos llegan a la solución pero quien realiza menos iteraciones es la eliminación gaussiana con pivote y el más rápido es la descomposición de Doolittle con pivote. Veamos ahora los métodos para el sistema $Bx=d$:

In [8]:
res(gaussElimin(B,d))

La solución del sistema es:
x 1 = 9.444444444444445
x 2 = 11.166666666666666
x 3 = -2.833333333333333
Se realizaron 6 iteraciones en 1.1920928955078125e-06 segundos.


In [9]:
res(gaussJordan(B,d))

La solución del sistema es:
x 1 = 9.444444444444445
x 2 = 11.166666666666666
x 3 = -2.833333333333333
Se realizaron 10 iteraciones en 1.430511474609375e-06 segundos.


In [10]:
res(LU(B,d))

La solución del sistema es:
x 1 = 9.444444444444445
x 2 = 11.166666666666666
x 3 = -2.833333333333333
Se realizaron 7 iteraciones en 7.152557373046875e-07 segundos.


In [11]:
res(choleski(B,d))

La solución del sistema es:
x 1 = 9.444444444444441
x 2 = 11.166666666666663
x 3 = -2.833333333333332
Se realizaron 14 iteraciones en 1.430511474609375e-06 segundos.


In [12]:
res(GaussPivot(B,d))

La solución del sistema es:
x 1 = 9.444444444444446
x 2 = 11.16666666666667
x 3 = -2.833333333333334
Se realizaron 10 iteraciones en 1.1920928955078125e-06 segundos.


In [13]:
res(JordanPivot(B,d))

La solución del sistema es:
x 1 = 9.444444444444446
x 2 = 11.16666666666667
x 3 = -2.833333333333334
Se realizaron 17 iteraciones en 9.5367431640625e-07 segundos.


In [14]:
res(LUPivot(B,d))

La solución del sistema es:
x 1 = 9.444444444444446
x 2 = 11.16666666666667
x 3 = -2.833333333333334
Se realizaron 15 iteraciones en 1.1920928955078125e-06 segundos.


Note que para este sistema todos los métodos analizados podrían ser usados y llegan a la respuesta con al menos una tolerancia de $10^{-12}$. El método con menos iteraciones es la eliminación gaussiana y el más rápido es la descomposición de Doolittle sin pivote. Analicemos ahora el sistema $Cx=d$:

In [15]:
res(gaussElimin(C,d))

La solución del sistema es:
x 1 = -13.222222222222221
x 2 = -14.333333333333334
x 3 = 5.666666666666666
Se realizaron 6 iteraciones en 7.152557373046875e-07 segundos.


In [16]:
res(gaussJordan(C,d))

La solución del sistema es:
x 1 = -13.222222222222221
x 2 = -14.333333333333334
x 3 = 5.666666666666666
Se realizaron 10 iteraciones en 1.430511474609375e-06 segundos.


In [17]:
res(LU(C,d))

La solución del sistema es:
x 1 = -13.222222222222221
x 2 = -14.333333333333334
x 3 = 5.666666666666666
Se realizaron 7 iteraciones en 1.6689300537109375e-06 segundos.


In [18]:
print(choleski(C,d))

La matriz no es definida positiva, no se puede realizar la descomposición de Choleski.


In [19]:
res(GaussPivot(C,d))

La solución del sistema es:
x 1 = -13.222222222222216
x 2 = -14.333333333333327
x 3 = 5.666666666666664
Se realizaron 10 iteraciones en 1.430511474609375e-06 segundos.


In [20]:
res(JordanPivot(C,d))

La solución del sistema es:
x 1 = -13.222222222222216
x 2 = -14.333333333333327
x 3 = 5.666666666666664
Se realizaron 17 iteraciones en 1.9073486328125e-06 segundos.


In [21]:
res(LUPivot(C,d))

La solución del sistema es:
x 1 = -13.222222222222216
x 2 = -14.333333333333327
x 3 = 5.666666666666664
Se realizaron 15 iteraciones en 1.6689300537109375e-06 segundos.


En este caso no es posible realizar la descomposición de Choleski ya que la matriz $C$ no es definida positiva, el resto de los métodos si es posible utilizarlos. La eliminación gaussiana realiza menos iteraciones y es el más rápido.
### Conclusión
Se realizó un experimento para medir y comparar la eficacia de la eliminación gaussiana con y sin pivote, la eliminación de Gauss-Jordan con y sin pivote, la descomposición de Doolittle con y sin pivote, y la descomposición de Choleski. Para el experimento se usaron tres sistemas cuyas matrices ampliadas tenían diferentes características con el fin de comparar adecuadamente los métodos.

Con el experimento se llegaron a las siguiente conclusiones:

-Los métodos sin pivote se encuentran límitados si en medio de la eliminación algún elemento de la diagonal se vuelve cero. Por lo que si no se conoce la naturaleza de la matriz es recomendable utilizar los métodos con pivote aunque se requieran más iteraciones o tiempo.

-La descomposición de Choleski también encuentra su limitación si la matriz no es definida positiva y siḿétrica, por lo que encuentra su utilidad cuando es requerida para ciertas aplicaciones particulares donde surgen matrices de la naturaleza adecuada para el método.  